In [ ]:
'''from google.colab import files
uploaded = files.upload()'''

'from google.colab import files\nuploaded = files.upload()'

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
PATH = '/content/drive/My Drive/NLP/'

In [ ]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
!pip install transformers
from transformers import *
import tokenizers
print('TF version',tf.__version__)
!pip install emoji
import emoji
import re
import math
import keras
import json
import string
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

     |████████████████████████████████| 675kB 5.4MB/s 
     |████████████████████████████████| 890kB 13.3MB/s 
     |████████████████████████████████| 3.8MB 34.5MB/s 
     |████████████████████████████████| 1.1MB 44.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0ba288e416affe8618b35bd77c01c11caa346430de826cb922521df132cd357f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
TF version 2.2.0
     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=4d2d2dea7b07d0f82ed58cb95397be42dbf190a23056fc6fcdaf8f024f2f76a2
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
with open(PATH+'categories.json') as f:
  categories = json.load(f)

tweets = []
for line in open(PATH+'train_gold.json', 'r'):
    tweets.append(json.loads(line))

test = []
for line in open(PATH+'test_unlabeled.json', 'r'):
    test.append(json.loads(line))
    
dev = []
for line in open(PATH+'dev_unlabeled.json','r'):
  dev.append(json.loads(line))

In [ ]:
f = open(PATH+"weird.txt")
weird = f.read().splitlines()
#print(weird)
f.close()

In [ ]:
def rule(token):
    '''
    Remove tokens with
    x. stop words  Ex. the, we (Update: only rmv '️')
    2. punctuation Ex. !!
    3. numbers     Ex. 1kg
    4. hashtags    Ex. #NLP
    5. tags        Ex. @hannnnk
    6. hyper link  Ex. https://...
    '''
    return True if token not in stopword and \
                   token not in punctuation and \
                   "\n" not in token and \
                   not re.search('\d+', token) and \
                   not token.startswith('#') and \
                   not token.startswith('@') and \
                   not token.startswith('https:') \
                   else False

In [ ]:
def clean(tweet):
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"here's", "here is", tweet)
    tweet = re.sub(r"today's", "today is", tweet)
    tweet = re.sub(r"ain't", "are not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"how's", "how is", tweet)
    tweet = re.sub(r"thet've", "they have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"ya'll", "you all", tweet)
    tweet = re.sub(r"i'm", "i am", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"i've", "i have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"who-the-hell-even-knows-anymore", "who the hell even knows anymore", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"she's", "she is", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"shouldn't", "shoud not", tweet)  
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"you'd", "you had", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"no-excuse", "no excuse", tweet)
    tweet = re.sub(r"vote-by-mail", "vote by mail", tweet)
    tweet = re.sub(r"votesafe", "vote safe", tweet)
    tweet = re.sub(r"lmaoooo", "laugh my ass off", tweet)
    tweet = re.sub(r"we'd", "we had", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"he'd", "he had", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"hadn't", "had not", tweet)
    tweet = re.sub(r"diefrom", "die from", tweet)
    tweet = re.sub(r"dababy", "rapper", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"testing-based", "testing based", tweet)
    tweet = re.sub(r"that'll", "that will", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"cashapp", "cash app", tweet)
    tweet = re.sub(r"brexit-induced", "brexit induced", tweet)
    tweet = re.sub(r"president-hating", "president hating", tweet)
    tweet = re.sub(r"picked-over", "picked over", tweet)
    
    tweet = re.sub(r"what’s", "what is", tweet)
    tweet = re.sub(r"we’ll", "we will", tweet)
    tweet = re.sub(r"who’s", "who is", tweet)
    tweet = re.sub(r"here’s", "here is", tweet)
    tweet = re.sub(r"today’s", "today is", tweet)
    tweet = re.sub(r"ain’t", "are not", tweet)
    tweet = re.sub(r"we’ve", "we have", tweet)
    tweet = re.sub(r"how’s", "how is", tweet)
    tweet = re.sub(r"thet’ve", "they have", tweet)
    tweet = re.sub(r"it’ll", "it will", tweet)
    tweet = re.sub(r"ya’ll", "you all", tweet)
    tweet = re.sub(r"i’m", "i am", tweet)
    tweet = re.sub(r"don’t", "do not", tweet)
    tweet = re.sub(r"it’s", "it is", tweet)
    tweet = re.sub(r"you’re", "you are", tweet)
    tweet = re.sub(r"can’t", "cannot", tweet)
    tweet = re.sub(r"i’ve", "i have", tweet)
    tweet = re.sub(r"that’s", "that is", tweet)
    tweet = re.sub(r"we’re", "we are", tweet)
    tweet = re.sub(r"i’ll", "I will", tweet)
    tweet = re.sub(r"he’s", "he is", tweet)
    tweet = re.sub(r"let’s", "let us", tweet)
    tweet = re.sub(r"didn’t", "did not", tweet)
    tweet = re.sub(r"y’all", "you all", tweet)
    tweet = re.sub(r"doesn’t", "does not", tweet)
    tweet = re.sub(r"i’d", "I would", tweet)
    tweet = re.sub(r"there’s", "there is", tweet)
    tweet = re.sub(r"isn’t", "is not", tweet)
    tweet = re.sub(r"won’t", "will not", tweet)
    tweet = re.sub(r"they’re", "they are", tweet)
    tweet = re.sub(r"haven’t", "have not", tweet)
    tweet = re.sub(r"couldn’t", "could not", tweet)
    tweet = re.sub(r"she’s", "she is", tweet)
    tweet = re.sub(r"aren’t", "are not", tweet)
    tweet = re.sub(r"you’ve", "you have", tweet)
    tweet = re.sub(r"you’ll", "you will", tweet)
    tweet = re.sub(r"wouldn’t", "would not", tweet)
    tweet = re.sub(r"wasn’t", "was not", tweet)
    tweet = re.sub(r"shouldn’t", "shoud not", tweet)
    tweet = re.sub(r"they’ll", "they will", tweet)
    tweet = re.sub(r"weren’t", "were not", tweet)
    tweet = re.sub(r"you’d", "you had", tweet)
    tweet = re.sub(r"they’ve", "they have", tweet)
    tweet = re.sub(r"we’d", "we had", tweet)
    tweet = re.sub(r"he’ll", "he will", tweet)
    tweet = re.sub(r"he’d", "he had", tweet)
    tweet = re.sub(r"would’ve", "would have", tweet)
    tweet = re.sub(r"hadn’t", "had not", tweet)
    tweet = re.sub(r"hasn’t", "has not", tweet)
    tweet = re.sub(r"that’ll", "that will", tweet)
    tweet = re.sub(r"should’ve", "should have", tweet)
    
    return tweet

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
tknzr = TweetTokenizer()

In [ ]:
stopword = ['️']
punctuation = string.punctuation+'“’”...—…‼‘'

In [ ]:
for t in tweets:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())
for t in dev:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())
for t in test:
    t['text'] = clean(t['text'].lower())
    t['reply'] = clean(t['reply'].lower())

In [ ]:
for t in tweets:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2

for t in dev:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2
    
for t in test:
    temp = [token.lower() for token in tknzr.tokenize(t['text']) if rule(token.lower())]
    temp2 = [token.lower() for token in tknzr.tokenize(t['reply']) if rule(token.lower())]

    t['text_clean'] = temp
    t['reply_clean'] = temp2

In [ ]:
'''
Deal with special words for embedding such as
covid  (806) → coronavirus
'''
special = {
    'covid':'coronavirus',
    "ventilators":"ventilator",
    "virologist":'virology',
    'fauci':'physician',
    "onlyfans":"app",
    "fortnite":"game",
    "valorant":"game",
    "jeudy":"athlete",
    "ruggs":"athlete",
    "tagovailoa":"athlete",
    "rashford":"athlete",
    "birx":"doctor",
    "blaqbonez":"rapper",
    'mcsally':"politician",
    "intubated":"intubation",
    "hydroxychloroquine":"medication",
    "today's":"today",
    "pubg":"game",
    "government's":'government',
    "man's":'man',
    "someone's":'someone',
    "country's":'country',
    "everyone's":'everyone',
    "people's":'people',
    "club's":'club',
    "china's":'china',
    "america's":'america',
    "world's":'world',
    "life's":'life',
    "earth's":'earth',
    "mom's":'mom',
    '<3':'❤',
    ':)':"🙂",
    ":(":"😞",
    "):":"😞",
    ":/":"😕",
    "pre-covid":'coronavirus',
    "strategise":"strategies",
    'stratagy':'strategy',
    "strategiser":"strategy",
    "stratagised":"strategy",
    'ohhhh':"ohhh",
    'yesssss':"yes",
    'youuu':'you',
    'sirrr':'sir',
    'ohhhhh':'ohhh',
    'youuuu':'you',
    'lmaooooo':'lamo',
    'guysss':'guys',
    'meeeee':'me',
    'yessss':'yes',
    'lieeeeeee':'lie',
    'youuuuu':'you',
    'yesssssss':'yes',
    'lollll':'lol',
    'plzzzz':'plz',
    'enddddddd':'end',
}

In [ ]:
emoji_remain_text = ['😭','❤','👏','😊','😔','🙏','🙃','💜','👍','🙌','😁','🤗','💔','☹','😎','😢','💛','💖','💰','💯','💗','💪','😞','😣',"🙂","😞","😕"]
emoji_remain_reply = ['❤','🙏','😭','👏','💜','👍','🤗','💕','💖','😘','💙','♥','💞','🙌','💗','😉','💛','🤦','💓','🖤','🙄','💯','☺','💃','🎉','😔','💚','💔','✨','😢','🌹',"🙂","😞","😕"]

In [ ]:
def rule_emoji_text(token, emoji_remain):
    if len(token)==1:
        if ord(token)>122:
            if token not in emoji_remain:
                return False
    return True

In [ ]:
tokens = Counter()
tokens2 = Counter()
for t in tweets:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    tokens.update(temp)
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2
    tokens2.update(temp2)
    
temp = pd.DataFrame(tokens.most_common(30))
temp.columns = ['tokens_main','count']
display(temp.style.background_gradient(cmap='Blues'))
temp2 = pd.DataFrame(tokens2.most_common(30))
temp2.columns = ['tokens_reply','count']
display(temp2.style.background_gradient(cmap='Reds'))

for t in dev:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2
    
for t in test:
    temp = [token for token in t['text_clean'] if rule_emoji_text(token,emoji_remain_text)]
    t['text_clean'] = temp
    temp2 = [token for token in t['reply_clean'] if rule_emoji_text(token,emoji_remain_reply)]
    for idx,token in enumerate(temp2):
        if token == '🥺':
            temp2[idx] = '❤'
    t['reply_clean'] = temp2

,tokens_main,count
0,i,22433
1,the,18746
2,to,17175
3,a,14900
4,and,12369
5,is,12307
6,you,11669
7,of,9100
8,my,7713
9,in,7540


,tokens_reply,count
0,i,3620
1,you,3411
2,the,2588
3,to,2197
4,is,2025
5,a,1952
6,and,1819
7,it,1562
8,not,1340
9,that,1256


In [ ]:
for t in tweets:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in dev:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]
for t in test:
    for idx,token in enumerate(t['text_clean']):
        if token in special:
            t['text_clean'][idx] = special[token]
        if token in emoji_remain_text:
            t['text_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    for idx,token in enumerate(t['reply_clean']):
        if token in special:
            t['reply_clean'][idx] = special[token]
        if token in emoji_remain_reply:
            t['reply_clean'][idx] = re.sub("_", " ",re.sub(':','',emoji.demojize(token)))
    #t['text_clean'] = [c for c in t['text_clean'] if c not in weird]
    #t['reply_clean'] = [c for c in t['reply_clean'] if c not in weird]

In [ ]:
cat_index = {}
for idx,c in enumerate(categories):
    cat_index[c]=idx
    
def cat2vec(cat):
    res = []
    for tweet in cat:
        temp = np.zeros(43)
        for c in tweet:
            temp[cat_index[c]] = 1
        res.append(temp)
    return np.array(res)

In [ ]:
tweets[25376]['text_clean'] = tweets[25376]['text_clean'][3:]
train_text = [" ".join(t['text_clean']) for t in tweets]
tweets[28767]['reply_clean'] = tweets[28767]['reply_clean'][:55]
train_reply = [" ".join(t['reply_clean']) for t in tweets]
train_y = [t['categories'] for t in tweets]


dev[1581]['text_clean'] = dev[1581]['text_clean'][20:]
dev_text = [" ".join(t['text_clean']) for t in dev]
dev_reply = [" ".join(t['reply_clean']) for t in dev]

test[1487]['text_clean'] = test[1487]['text_clean'][20:]
test[2949]['reply_clean'] = test[2949]['reply_clean'][:9]
test_text = [" ".join(t['text_clean']) for t in test]
test_reply = [" ".join(t['reply_clean']) for t in test]

In [ ]:
train_y = cat2vec(train_y)
train_y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [ ]:
train_dict = {}
for i in range(32000):
  temp = train_text[i]+train_reply[i]
  if temp in train_dict:
    train_dict[temp].append((train_y[i],i))
  else:
    train_dict[temp] = [(train_y[i],i)]

duplicated = []
for key in train_dict:
  if len(train_dict[key])>1:
    for id in train_dict[key][1:]:
      duplicated.append(id[1])

new = []
for key in train_dict:
  if len(train_dict[key])>1:
    first_idx = train_dict[key][0][1]
    temp = np.zeros(43)
    for id in train_dict[key]:
      temp = np.logical_or(temp, id[0])
    temp = temp.astype(int)
    new.append((first_idx,temp))

In [ ]:
for idx, cat in new:
  train_y[idx] = cat

In [ ]:
print('before: ',len(train_text), len(train_reply), len(train_y))
_tweets = np.delete(tweets, duplicated)
train_text = np.delete(train_text, duplicated)
train_reply = np.delete(train_reply, duplicated)
train_y = np.delete(train_y, duplicated, axis=0)
print('after: ',len(train_text), len(train_reply), len(train_y), len(_tweets))

before:  32000 32000 32000
after:  26967 26967 26967 26967


In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_x = []
for i in range(len(train_text)):
  train_x.append(tokenizer.encode(train_text[i], train_reply[i],skip_special_tokens=True))

dev_x = []
for i in range(len(dev_text)):
  dev_x.append(tokenizer.encode(dev_text[i], dev_reply[i],skip_special_tokens=True))

test_x = []
for i in range(len(test_text)):
  test_x.append(tokenizer.encode(test_text[i], test_reply[i],skip_special_tokens=True))

In [ ]:
print(len(max(train_x,key=len)))
print(len(max(dev_x,key=len)))
print(len(max(test_x,key=len)))

160
122
139


In [ ]:
gg = []
for idx,t in enumerate(train_x):
  gg.append((len(t),idx))
sorted(gg,key=lambda x:x[0],reverse=True)[:10]

[(160, 9241),
 (139, 15112),
 (139, 21609),
 (132, 24351),
 (130, 22963),
 (128, 16528),
 (128, 26300),
 (127, 13504),
 (126, 12215),
 (126, 24434)]

In [ ]:
MAX_LEN=165

In [ ]:
attention_mask = []
for t in train_x:
  N = len(t)
  attention_mask.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

attention_mask_d = []
for t in dev_x:
  N = len(t)
  attention_mask_d.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

attention_mask_t = []
for t in test_x:
  N = len(t)
  attention_mask_t.append(list(np.ones(N, dtype=int))+list(np.zeros(MAX_LEN-N, dtype=int)))

In [ ]:
attention_mask = np.mat(attention_mask)
attention_mask_d = np.mat(attention_mask_d)
attention_mask_t = np.mat(attention_mask_t)

In [ ]:
train_x = pad_sequences(train_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 0)
dev_x = pad_sequences(dev_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 0)
test_x = pad_sequences(test_x,  maxlen=MAX_LEN, dtype='int', padding='post', value = 0)

In [ ]:
token_type_ids = np.zeros((train_x.shape[0],MAX_LEN))
token_type_ids_d = np.zeros((dev_x.shape[0],MAX_LEN))
token_type_ids_t = np.zeros((test_x.shape[0],MAX_LEN))

## Build BERT Model

In [ ]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = BertConfig.from_pretrained('bert-base-uncased', num_labels=43)
    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

In [ ]:
def build_model_large():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = BertConfig.from_pretrained('bert-large-uncased', num_labels=43)
    bert_model = TFBertForSequenceClassification.from_pretrained('bert-large-uncased',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

In [ ]:
def build_model_wwm():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = BertConfig.from_pretrained('bert-large-uncased-whole-word-masking', num_labels=43)
    bert_model = TFBertForSequenceClassification.from_pretrained('bert-large-uncased-whole-word-masking',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Dense(43)(x1)
    x1 = tf.keras.layers.Activation('sigmoid')(x1)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=x1)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

### BERT(Base)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
model = build_model()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=5, batch_size=32, verbose=1)

### BERT (Large)
> MAX_LEN=140, batchsize=16



In [ ]:
model_large = build_model_large()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

model_large.fit([train_x, attention_mask, token_type_ids], train_y, epochs=4, batch_size=16, verbose=1)


Epoch 1/4
1674/1674 [==============================] - 1406s 840ms/step - loss: 0.1911
Epoch 2/4
1674/1674 [==============================] - 1405s 839ms/step - loss: 0.1599
Epoch 3/4
1674/1674 [==============================] - 1405s 839ms/step - loss: 0.1557
Epoch 4/4
1674/1674 [==============================] - 1405s 839ms/step - loss: 0.1536


### Pred and Save

In [ ]:
pred = model_large.predict([dev_x, attention_mask_d, token_type_ids_d])
pred_t = model_large.predict([test_x, attention_mask_t, token_type_ids_t])

In [ ]:
with open(PATH+'pred_bert_large_uncased_d.npy', 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_bert_large_uncased_t.npy', 'wb') as f:
    np.save(f, pred_t)

## Ensemble BERT(Base)
* 0-10: batchsize 16 epoch 5
* batchsize 16, epoch 8 ->worse

In [ ]:
start = 11
for i in range(start,start+10):
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  model = build_model()
  model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=5, batch_size=16, verbose=1)
  pred = model.predict([dev_x, attention_mask_d, token_type_ids_d])
  pred_t = model.predict([test_x, attention_mask_t, token_type_ids_t])
  with open(PATH+'pred_bert_uncased{}_d.npy'.format(i), 'wb') as f:
    np.save(f, pred)

  with open(PATH+'pred_bert_uncased{}_t.npy'.format(i), 'wb') as f:
    np.save(f, pred_t)
  del model


Epoch 1/8
1686/1686 [==============================] - 596s 353ms/step - loss: 0.1952
Epoch 2/8
1686/1686 [==============================] - 596s 353ms/step - loss: 0.1516
Epoch 3/8
1686/1686 [==============================] - 596s 353ms/step - loss: 0.1416
Epoch 4/8
1686/1686 [==============================] - 596s 354ms/step - loss: 0.1319
Epoch 5/8
1686/1686 [==============================] - 596s 354ms/step - loss: 0.1197
Epoch 6/8
1686/1686 [==============================] - 596s 353ms/step - loss: 0.1069
Epoch 7/8
1686/1686 [==============================] - 598s 354ms/step - loss: 0.0954
Epoch 8/8
1686/1686 [==============================] - 598s 354ms/step - loss: 0.0853
Epoch 1/8
1686/1686 [==============================] - 596s 354ms/step - loss: 0.2054
Epoch 2/8
1686/1686 [==============================] - 597s 354ms/step - loss: 0.1511
Epoch 3/8
1686/1686 [==============================] - 596s 353ms/step - loss: 0.1379
Epoch 4/8
1686/1686 [==============================] 

In [ ]:
pred = []
pred_t = []
start = 16
end = 20
for i in range(start, end+1):
  with open(PATH+'pred_bert_uncased{}_d.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred.append(a)
  with open(PATH+'pred_bert_uncased{}_t.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred_t.append(a)
pred = np.mean(pred,axis=0)
pred_t = np.mean(pred_t,axis=0)

with open(PATH+'pred_bert_uncased_ensemble{}-{}_d.npy'.format(start,end), 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_bert_uncased_ensemble{}-{}_t.npy'.format(start,end), 'wb') as f:
    np.save(f, pred_t)

"with open(PATH+'pred_bert_uncased_ensemble{}-{}_d.npy'.format(start,end), 'wb') as f:\n    np.save(f, pred)\n\nwith open(PATH+'pred_bert_uncased_ensemble{}-{}_t.npy'.format(start,end), 'wb') as f:\n    np.save(f, pred_t)"

## Ensemble BERT(Large-WWM)

In [ ]:
start = 0
for i in range(start,start+10):
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  model = build_model_wwm()
  model.fit([train_x, attention_mask, token_type_ids], train_y, epochs=5, batch_size=16, verbose=1)
  pred = model.predict([dev_x, attention_mask_d, token_type_ids_d])
  pred_t = model.predict([test_x, attention_mask_t, token_type_ids_t])
  with open(PATH+'pred_bert_large_wwm_uncased{}_d.npy'.format(i), 'wb') as f:
    np.save(f, pred)

  with open(PATH+'pred_bert_large_wwm_uncased{}_t.npy'.format(i), 'wb') as f:
    np.save(f, pred_t)
  del model

In [ ]:
pred = []
pred_t = []
start = 0
end = 10
for i in range(start, end+1):
  with open(PATH+'pred_bert_large_wwm_uncased{}_d.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred.append(a)
  with open(PATH+'pred_bert_large_wwm_uncased{}_t.npy'.format(i), 'rb') as f:
    a = np.load(f)
    pred_t.append(a)
pred = np.mean(pred,axis=0)
pred_t = np.mean(pred_t,axis=0)

with open(PATH+'pred_bert_large_wwm_uncased_ensemble{}-{}_d.npy'.format(start,end), 'wb') as f:
    np.save(f, pred)

with open(PATH+'pred_bert_large_wwm_uncased_ensemble{}-{}_t.npy'.format(start,end), 'wb') as f:
    np.save(f, pred_t)

## Get Result

In [ ]:
def decode_cat(res):
    ans = []
    for r in res:
        temp = [categories[i] for i in r]
        ans.append(temp)
    return ans

In [ ]:
results = [np.argsort(res)[::-1][:6] for res in pred]
results = decode_cat(results)

In [ ]:
for idx,t in enumerate(dev):
    t['categories'] = results[idx]

In [ ]:
dev2 = []
for line in open(PATH+'dev_unlabeled.json','r'):
    dev2.append(json.loads(line))
for idx,d in enumerate(dev):
    dev2[idx]['categories']=d['categories']

In [ ]:
for i in dev2:
  with open('dev.json', 'a', encoding='utf-8') as f:
    json.dump(i, f, ensure_ascii=False)
    f.write('\n')